## Initialize the Agent

We are initializing a Llama Stack application which starts a Llama Stack server running alongside our main application. Running a separate Llama Stack server is a better approach for real use cases, but for our simple example this setup is sufficient.

On the other hand, we are going to use the model llama3.2:3b-instruct-fp16 served by a local Ollama instance. We need to define a set of rules or instructions so that the LLM can correctly infer when and how to use the tools we provide. We also need to configure the temperature parameter, which controls the balance between strictness and creativity of the model.

Finally, we add our tools, which are implemented in the files [math.py](./tools/maths.py) and [uppercase.py](./tools/uppercaser.py).

In [6]:
import os

from llama_stack_client import Agent, AgentEventLogger
from llama_stack.distribution.library_client import LlamaStackAsLibraryClient
from tools.uppercaser import to_uppercase
from tools.maths import add, divide, multiply, subtract


model = "llama3.2:3b-instruct-fp16"
os.environ["INFERENCE_MODEL"] = model
client = LlamaStackAsLibraryClient('ollama')
client.initialize()

instructions = """
You are a smart assistant that helps users with his questions.
You have differents available tools and you must use to retrieve information or perform actions.
You must not generate code or scripts.
No simulate the output, perform all operations with tools available.
Multiple tool calls may need to happen serially.
You may only call one tool at a time.
Remember you may need to pass the output of the first tool to subsequent tool calls in some cases
"""

agent = Agent(
  client,
  model=model,
  instructions=instructions,
  sampling_params={
    "strategy": {"type": "top_p", "temperature": 0.2, "top_p": 0.9},
  },
  tools=[
      to_uppercase, add, divide, multiply, subtract
  ],
)
session_id = agent.create_session('new_session')

Using config ollama:

apis:
- agents
- datasetio
- eval
- inference
- post_training
- safety
- scoring
- telemetry
- tool_runtime
- vector_io
benchmarks: []
container_image: null
datasets: []
external_providers_dir: null
image_name: ollama
inference_store:
  db_path: /home/jbarea/.llama/distributions/ollama/inference_store.db
  type: sqlite
logging: null
metadata_store:
  db_path: /home/jbarea/.llama/distributions/ollama/registry.db
  namespace: null
  type: sqlite
models:
- metadata: {}
  model_id: llama3.2:3b-instruct-fp16
  model_type: !!python/object/apply:llama_stack.apis.models.models.ModelType
  - llm
  provider_id: ollama
  provider_model_id: null
- metadata:
    embedding_dimension: 384
  model_id: all-MiniLM-L6-v2
  model_type: !!python/object/apply:llama_stack.apis.models.models.ModelType
  - embedding
  provider_id: ollama
  provider_model_id: all-minilm:latest
providers:
  agents:
  - config:
      persistence_store:
        db_path: /home/jbarea/.llama/distributions/ollama/agents_store.db
        namespace: null
        type: sqlite
      responses_store:
        db_path: /home/jbarea/.llama/distributions/ollama/responses_store.db
        type: sqlite
    provider_id: meta-reference
    provider_type: inline::meta-reference
  datasetio:
  - config:
      kvstore:
        db_path: /home/jbarea/.llama/distributions/ollama/huggingface_datasetio.db
        namespace: null
        type: sqlite
    provider_id: huggingface
    provider_type: remote::huggingface
  - config:
      kvstore:
        db_path: /home/jbarea/.llama/distributions/ollama/localfs_datasetio.db
        namespace: null
        type: sqlite
    provider_id: localfs
    provider_type: inline::localfs
  eval:
  - config:
      kvstore:
        db_path: /home/jbarea/.llama/distributions/ollama/meta_reference_eval.db
        namespace: null
        type: sqlite
    provider_id: meta-reference
    provider_type: inline::meta-reference
  inference:
  - config:
      url: http://localhost:11434
    provider_id: ollama
    provider_type: remote::ollama
  post_training:
  - config:
      checkpoint_format: huggingface
      device: cpu
      distributed_backend: null
    provider_id: huggingface
    provider_type: inline::huggingface
  safety:
  - config:
      excluded_categories: []
    provider_id: llama-guard
    provider_type: inline::llama-guard
  scoring:
  - config: {}
    provider_id: basic
    provider_type: inline::basic
  - config: {}
    provider_id: llm-as-judge
    provider_type: inline::llm-as-judge
  - config:
      openai_api_key: '********'
    provider_id: braintrust
    provider_type: inline::braintrust
  telemetry:
  - config:
      service_name: ''
      sinks: sqlite
      sqlite_db_path: /home/jbarea/.llama/distributions/ollama/trace_store.db
    provider_id: meta-reference
    provider_type: inline::meta-reference
  tool_runtime:
  - config:
      api_key: '********'
      max_results: 3
    provider_id: brave-search
    provider_type: remote::brave-search
  - config:
      api_key: '********'
      max_results: 3
    provider_id: tavily-search
    provider_type: remote::tavily-search
  - config: {}
    provider_id: rag-runtime
    provider_type: inline::rag-runtime
  - config: {}
    provider_id: model-context-protocol
    provider_type: remote::model-context-protocol
  - config:
      api_key: '********'
    provider_id: wolfram-alpha
    provider_type: remote::wolfram-alpha
  vector_io:
  - config:
      kvstore:
        db_path: /home/jbarea/.llama/distributions/ollama/faiss_store.db
        namespace: null
        type: sqlite
    provider_id: faiss
    provider_type: inline::faiss
scoring_fns: []
server:
  auth: null
  host: null
  port: 8321
  quota: null
  tls_cafile: null
  tls_certfile: null
  tls_keyfile: null
shields: []
tool_groups:
- args: null
  mcp_endpoint: null
  provider_id: tavily-search
  toolgroup_id: builtin::websearch
- args: null
  mcp_endpoint: null
  provider_id: rag-runtime
  toolgroup_id: builtin::rag
- args: null
  mcp_endpoi

## Interaction of model with tools

Now we can ask the model to perform tasks related to the tools. For example, if the input contains keywords related to the provided tools, the model will infer that it needs to use those specific tools.

If we ask to uppercase some text, the presence of the word "uppercase" signals that it should use the `to_uppercase` tool. Additionally, if the input involves multiple tasks requiring different tools, the model should call each tool sequentially and use the output of the previous tool as the input for the next one. This is not a default behavior, but we define it explicitly in the model instructions:
"Remember you may need to pass the output of the first tool to subsequent tool calls in some cases."

In [2]:


prompt = "Can you to uppercase 'hello world!'?"
print("prompt>", prompt)

response = agent.create_turn(
  messages=[{"role": "user", "content": prompt}],
  session_id=session_id,
)

for log in AgentEventLogger().log(response):
    log.print()

prompt> Can you to uppercase 'hello world!'?
inference> [to_uppercase(text='hello world!')]
tool_execution> Tool:to_uppercase Args:{'text': 'hello world!'}
tool_execution> Tool:to_uppercase Response:"HELLO WORLD!"
inference> How can I assist you further?


In [3]:
prompt = "First, add 3 + 3. Then divide the result by 2. Finally, multiply that result by 6. What is the final answer?"
print("prompt>", prompt)

response = agent.create_turn(
  messages=[{"role": "user", "content": prompt}],
  session_id=session_id,
)

for log in AgentEventLogger().log(response):
    log.print()

prompt> First, add 3 + 3. Then divide the result by 2. Finally, multiply that result by 6. What is the final answer?
inference> [add(a=3, b=3), divide(result=add(a=3, b=3), b=2), multiply(result=divide(a=3, b=3), b=6)]
tool_execution> Tool:add Args:{'a': 3.0, 'b': 3.0}
tool_execution> Tool:add Response:6
inference> [divide(a=6, b=2)]
tool_execution> Tool:divide Args:{'a': 6.0, 'b': 2.0}
tool_execution> Tool:divide Response:3.0
inference> [multiply(a=3.0, b=6)]
tool_execution> Tool:multiply Args:{'a': 3.0, 'b': 6.0}
tool_execution> Tool:multiply Response:18.0
inference> The final answer is 18.0.


## Using MCP server to provide tools to Agent


Now we will create a new agent that uses an MCP server to register tools. These tools are the same as the ones we used in the previous section, but this time they are provided via the MCP server.

Using an MCP server allows serving tools over HTTP to connected clients. This enables centralization of tools alongside different AI applications.

To do this, we need to run our MCP server, which is implemented with FastMCP and can be found in [mcp_server](./mcp_server.py). To start the server, simply run: `python mcp_server.py`

When you execute the script below, you will see the tools registered—the default tools and those served by the MCP server. Finally, we can query the model with the same prompt as before, and the output should be the same.


In [4]:
from llama_stack_client.types.toolgroup_register_params import McpEndpoint
from pprint import pprint

client.toolgroups.register(
    toolgroup_id="mcp::custom",
    provider_id="model-context-protocol",
    mcp_endpoint=McpEndpoint(uri="http://127.0.0.1:5000/sse"),
)

mcp_agent = Agent(
  client,
  model=model,
  instructions=instructions,
  tools=["mcp::custom"],
)


tools = client.tools.list()
pprint(tools)

[Tool(description='Insert documents into memory', identifier='insert_into_memory', parameters=[], provider_id='rag-runtime', tool_host=None, toolgroup_id='builtin::rag', type='tool', metadata=None, provider_resource_id=None),
 Tool(description='Search for information in a database.', identifier='knowledge_search', parameters=[Parameter(description='The query to search for. Can be a natural language sentence or keywords.', name='query', parameter_type='string', required=True, default=None)], provider_id='rag-runtime', tool_host=None, toolgroup_id='builtin::rag', type='tool', metadata=None, provider_resource_id=None),
 Tool(description='Search the web for information', identifier='web_search', parameters=[Parameter(description='The query to search for', name='query', parameter_type='string', required=True, default=None)], provider_id='tavily-search', tool_host=None, toolgroup_id='builtin::websearch', type='tool', metadata=None, provider_resource_id=None),
 Tool(description='Query Wolfram

In [5]:
prompts = [
    "Can you to uppercase 'hello world!'?",
    "First, add 3 + 3. Then divide the result by 2. Finally, multiply that result by 6. What is the final answer?",
]

mcp_session_id = mcp_agent.create_session('new_session')

for prompt in prompts:
    response = mcp_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=mcp_session_id,
    )

    print("prompt>", prompt)
    for log in AgentEventLogger().log(response):
        log.print()

prompt> Can you to uppercase 'hello world!'?
inference> [call_to_uppercase(text='hello world!')]
tool_execution> Tool:call_to_uppercase Args:{'text': 'hello world!'}
tool_execution> Tool:call_to_uppercase Response:[TextContentItem(text='{\n  "success": true,\n  "result": "HELLO WORLD!"\n}', type='text')]
inference> How can I assist you further?
prompt> First, add 3 + 3. Then divide the result by 2. Finally, multiply that result by 6. What is the final answer?
inference> [call_add(a=3, b=3, session_id="1234"), call_divide(a=9, b=2, session_id="1234"), call_multiply(a=54, session_id="1234")]
tool_execution> Tool:call_add Args:{'a': 3.0, 'b': 3.0, 'session_id': '1234'}
tool_execution> Tool:call_add Response:[TextContentItem(text='{\n  "success": true,\n  "result": 6\n}', type='text')]
inference> [call_divide(a=6, b=2, session_id="1234")]
tool_execution> Tool:call_divide Args:{'a': 6.0, 'b': 2.0, 'session_id': '1234'}
tool_execution> Tool:call_divide Response:[TextContentItem(text='{\n  "s